In [ ]:
!git clone https://github.com/aiformankind/wildfire-smoke-dataset.git

Cloning into 'wildfire-smoke-dataset'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 44 (delta 11), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.utils.data as data
import torchvision.utils as vutils
from torch.autograd import Variable
import numpy as np
import pandas as pd
import torch.optim as optim
import PIL.Image as Image

In [ ]:
!tar -xf drive/MyDrive/day_time_wildfire_v2.tar.gz

In [ ]:
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dset = dsets.ImageFolder("day_time_wildfire_v2", transform=transform)

In [ ]:
BATCH_SIZE=32


In [ ]:
dataloader = data.DataLoader(dset, shuffle=True, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

cuda:0


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.network = nn.Sequential(nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False), 
                                 nn.BatchNorm2d(512),
                                 nn.ReLU(True),
                                 nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
                                 nn.BatchNorm2d(256),
                                 nn.ReLU(True),
                                 nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(True),
                                 nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(True),
                                 nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
                                 nn.Tanh()).to(device)
  def forward(self, input):
    output = self.network(input)
    return output

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.network = nn.Sequential(nn.Conv2d(3, 64, 4, 2, 1, bias=False),
                                 nn.LeakyReLU(0.2, True),
                                 nn.Conv2d(64, 128, 4, 2, 1, bias=False),
                                 nn.BatchNorm2d(128),
                                 nn.LeakyReLU(0.2, True),
                                 nn.Conv2d(128, 256, 4, 2, 1, bias=False),
                                 nn.BatchNorm2d(256),
                                 nn.LeakyReLU(0.2, True),
                                 nn.Conv2d(256, 512, 4, 2, 1,bias=False),
                                 nn.BatchNorm2d(512),
                                 nn.LeakyReLU(0.2, True),
                                 nn.Conv2d(512, 1, 4, 2, 0, bias=False),
                                 nn.Sigmoid(),
                                 ).to(device)
  def forward(self, input):
    output = self.network(input).to(device)
    return output.view(-1) 
    

In [ ]:
discriminator = Discriminator()
generator = Generator()
discriminator.apply(weights_init)
generator.apply(weights_init)
criterion = nn.BCELoss().to(device)
optimizerDiscriminator = optim.Adam(discriminator.parameters(), lr=0.0001)
optimizerGenerator = optim.Adam(generator.parameters(), lr=0.0001)

In [ ]:
lossGOverTime = []
lossDOverTime = []
for epoch in range(190, 300):
  for (i, data) in enumerate(dataloader):
    discriminator.zero_grad()
    
    real, _ = data
    input = Variable(real).to(device)
    target = Variable(torch.ones((real.shape[0]))).to(device)
    output = discriminator(input)
    lossDReal = criterion(output, target)

    noise = Variable(torch.randn((real.shape[0], 100, 1, 1))).to(device)
    fake = generator(noise)
    target = torch.zeros((real.shape[0])).to(device)
    output = discriminator(fake.detach())
    lossDFake = criterion(output, target)

    lossD = lossDReal + lossDFake
    lossD.backward()
    optimizerDiscriminator.step()

    generator.zero_grad()

    target = Variable(torch.ones((real.shape[0]))).to(device)
    output = discriminator(fake)
    lossG = criterion(output, target)
    lossG.backward()
    optimizerGenerator.step()

    print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), lossD.data, lossG.data)) 
    if i % 100 == 0: # Every 100 steps:
      lossDOverTime.append(lossD.data)
      lossGOverTime.append(lossG.data)
      vutils.save_image(real, 'drive/MyDrive/SmokeGAN/real_samples.png', normalize = True) 
      fake = generator(noise) 
      vutils.save_image(fake.data, 'drive/MyDrive/SmokeGAN/128fake_samples_epoch_%03d.png' % (epoch), normalize = True)






Streaming output truncated to the last 5000 lines.
[227/25][37/69] Loss_D: 0.0015 Loss_G: 12.9477
[227/25][38/69] Loss_D: 0.0194 Loss_G: 13.0114
[227/25][39/69] Loss_D: 0.0025 Loss_G: 11.5157
[227/25][40/69] Loss_D: 0.0003 Loss_G: 11.3723
[227/25][41/69] Loss_D: 0.0056 Loss_G: 9.6483
[227/25][42/69] Loss_D: 0.0012 Loss_G: 12.1004
[227/25][43/69] Loss_D: 0.0069 Loss_G: 9.8288
[227/25][44/69] Loss_D: 0.0064 Loss_G: 8.5735
[227/25][45/69] Loss_D: 0.0004 Loss_G: 8.8966
[227/25][46/69] Loss_D: 0.0004 Loss_G: 8.8309
[227/25][47/69] Loss_D: 0.0104 Loss_G: 5.7643
[227/25][48/69] Loss_D: 0.0034 Loss_G: 7.4587
[227/25][49/69] Loss_D: 0.0048 Loss_G: 6.9359
[227/25][50/69] Loss_D: 0.0117 Loss_G: 7.6163
[227/25][51/69] Loss_D: 0.0018 Loss_G: 7.8228
[227/25][52/69] Loss_D: 0.0024 Loss_G: 7.4566
[227/25][53/69] Loss_D: 0.0081 Loss_G: 6.1763
[227/25][54/69] Loss_D: 0.0026 Loss_G: 7.8984
[227/25][55/69] Loss_D: 0.0001 Loss_G: 10.3400
[227/25][56/69] Loss_D: 0.0003 Loss_G: 9.3055
[227/25][57/69] Loss_D:

In [ ]:
noise = Variable(torch.randn((1, 100, 1, 1))).to(device)
fake = generator(noise)
vutils.save_image(fake.data, 'image.jpg', normalize = True)



In [ ]:
discriminator_state_dict = torch.load("drive/MyDrive/SmokeGAN/discriminator")
discriminator.load_state_dict(discriminator_state_dict)

AttributeError: ignored

In [ ]:
torch.save(discriminator.state_dict(), "drive/MyDrive/SmokeGAN/discriminator2")
torch.save(generator.state_dict(), "drive/MyDrive/SmokeGAN/generator2")
torch.save(optimizerDiscriminator.state_dict(), "drive/MyDrive/SmokeGAN/optimDiscriminator2")
torch.save(optimizerGenerator.state_dict(), "drive/MyDrive/SmokeGAN/optimGenerator2")

ValueError: ignored